# Initialize the session

We first need to initialize the session by:

* Mounting Google drive
* Importing (and installing) libraries
* Authenticating the Earth Engine session


### Mount Google drive to access modules

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# import
import sys
sys.path.append('/content/drive/MyDrive/_Research projects/asiaAg/pyscripts')


Mounted at /content/drive


### Import libraries and install `geemap`, if necessary

In [2]:
# %% capture
import importlib
import ee

try:
  import geemap
  print('Imported geemap')

except:
  try:
    import google.colab
    IN_COLAB = True
    print('In google colab')
    
  except:
    IN_COLAB = False
    print('Not in google colab')

  if IN_COLAB:
    print('Installing geemap...')
    !pip install geemap
    import geemap


In google colab
Installing geemap...
     |████████████████████████████████| 2.0 MB 5.2 MB/s 
     |████████████████████████████████| 187 kB 64.1 MB/s 
     |████████████████████████████████| 98 kB 7.1 MB/s 
     |████████████████████████████████| 99 kB 8.1 MB/s 
     |████████████████████████████████| 1.2 MB 42.3 MB/s 
     |████████████████████████████████| 8.8 MB 22.2 MB/s 
     |████████████████████████████████| 3.3 MB 39.8 MB/s 
     |████████████████████████████████| 3.0 MB 34.4 MB/s 
     |████████████████████████████████| 1.3 MB 55.0 MB/s 
     |████████████████████████████████| 46 kB 2.7 MB/s 
     |████████████████████████████████| 130 kB 47.4 MB/s 
     |████████████████████████████████| 95 kB 3.7 MB/s 
     |████████████████████████████████| 128 kB 40.9 MB/s 
     |████████████████████████████████| 342 kB 56.1 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 428 kB 58.6 MB/s 
     |████████████████████████████████| 130 kB

### Authenticate and initialize the Earth Engine session

In [3]:
import collections
collections.Callable = collections.abc.Callable
try:
  ee.Initialize()
  print('ee authenticated and initialized')
except:
  ee.Authenticate()
  ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=T1Lvp4pHJnsd9b16JbE4CGQfy8073OwZGrNMoUzTvEI&tc=MwNpTa7ibvujCJyMEWCrD6Xy5L2rfs7nbwUzJNFO1qo&cc=ODs_lNaN7RlnpisSThRWGF6pPOzImn4b0C6YZCoe7Ds

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiWusCKSuGjRqatSx4nzZBTJ6P2cTsqkOZHdLenOlQ3Fs5ptfx13MU

Successfully saved authorization token.


In [4]:
import ees
import rs
import os
import numpy as np

# Retrieve the watershed

In [6]:
# watershed_pt = ee.Geometry.Point([73.137, 34.65]) # Indus basin
watershed_pt = ee.Geometry.Point([76, 12.9]) # Cauvery basin

hyd_watershed = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_4') \
  .filterBounds(watershed_pt) \
  .union()

# # Map the watershed
# basmap = geemap.Map()
# basmap.addLayer(hyd_watershed,{},'watershed')
# basmap.centerObject(hyd_watershed,7)
# basmap

# Prepare / sample / output Sentinel-2

## Prepare Sentinel-2 Surface Reflectance

The next step will be to prepare Sentinel-2 data for export. A few tasks need to be completed:

1. Import the image collection and add cloud mask
2. Generate sampling locations
3. For each sampling location, generate a grid of pixels, loop through images and extract sample, and export the data

### 1. Import image collection and add cloud mask

This step uses two functions from the `ees.py` module. 

* First, `get_s2_sr_cld_col` is used to retrieve Sentinel-2 reflectance and Sentinel-2 cloud probability and join the two image collections over the dates specified in `s2params`.
* Second, `add_cld_shadow_mask_func` is used to generate the cloud + shadow mask for each image in the collection.

The last step in this cell is to use the `mosaic` function to generate an image for mapping and testing. Note that the `reproject` allows the function `sampleRectangle` to work properly.

In [7]:
# importlib.reload(ees)
date_range = ['2014-01-01', '2022-04-30']
# date_range = ['2020-01-01', '2020-01-30'] # TESTING / DEBUGGING

s1_ic = ee.ImageCollection("COPERNICUS/S1_GRD") \
  .filterBounds(hyd_watershed) \
  .filterDate(date_range[0],date_range[1])
s1_im = s1_ic.mosaic()

# params variable is used to pass  information to the cloud masking functions.
# see help(add_cld_shadow_mask_func)
s2params = {
    'START_DATE' : date_range[0],
    'END_DATE' : date_range[1],
    'CLOUD_FILTER' : 50,
    'CLD_PRB_THRESH' : 53, # 53 for Cauvery # 55 for Indus
    'NIR_DRK_THRESH' : 0.2,
    'CLD_PRJ_DIST' : 1,
    'BUFFER' : 50
}

s2_clouds_ic = ees.get_s2_sr_cld_col(hyd_watershed, s2params) \
  .map(ees.add_cld_shadow_mask_func(s2params))

# For some reason the reproject() works so that subsequent sampling returns the whole rectangular array
# see https://stackoverflow.com/questions/64012752/gee-samplerectangle-returning-1x1-array
s2_clouds_im = s2_clouds_ic.mosaic().reproject(crs = ee.Projection('EPSG:4326'), scale=10) #.clip(hyd_watershed)

### 2. Generate sampling regions

* Use `image.Sample()` to generate sampling points.

In [8]:
# Map.add_interaction(mapwidgets.
output_bands = ['B8','B4','B3','B2','clouds','cloudmask','shadows','probability']
s1_bands = ['HH','VV','HV','VH','angle']

numLocations = 22

s2_sample_locs = s2_clouds_im.sample(
    region = hyd_watershed, 
    scale = 10,
    numPixels = numLocations, seed = 10, 
    geometries = True).map(rs.set_feature_id_func('loc_id')).select('loc_id')


Generate rectangles around training data

In [37]:
# These parameters determine the number of pixels and the spacing for the grid
# of pixels to be sampled.
square_px = 10
nominal_scale = 200 # this is the width of a pixel in the image

# Extract rectangles around sampling locations
def buffer_rect(pt):
  return ee.Feature(pt.geometry().buffer(256 / 2 * square_px).bounds())
s2_sample_rects_all = s2_sample_locs.map(buffer_rect)

In [65]:
s1_im.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'precision': 'double', 'type': 'PixelType'},
   'id': 'VV'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'precision': 'double', 'type': 'PixelType'},
   'id': 'VH'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'precision': 'float', 'type': 'PixelType'},
   'id': 'angle'}],
 'type': 'Image'}

In [ ]:
array_empty_plus = [[0, 1, 0],
                    [1, 0, 1],
                    [0, 1, 0]]
empty_plus = ee.Kernel.fixed(weights = array_empty_plus, normalize = True)
# kernel_square = ee.Kernel.square(radius = 3)
# kernel_square.getInfo()
s1_conv = s1_im.convolve(kernel = empty_plus).select(['VV','VH'],['VV_ep','VH_ep'])
# Map.addLayer(s1_conv.clip(l1), {'bands':['VV'], 'min':-15, 'max':15}, 'S1 conv')

sentinel_im = s2_clouds_im.addBands(s1_im.multiply(1e4)).addBands(s1_conv.multiply(1e4)).int()

# sentinel_im.getInfo()

In [36]:
kernel_square.getInfo()

{'center': [1, 1],
 'radius': 1,
 'type': 'Kernel.square',
 'weights': '\n  [0.1111111111111111, 0.1111111111111111, 0.1111111111111111]\n  [0.1111111111111111, 0.1111111111111111, 0.1111111111111111]\n  [0.1111111111111111, 0.1111111111111111, 0.1111111111111111]'}

In [19]:
Map = geemap.Map()
l1 = s2_sample_rects_all.first()
Map.centerObject(l1, 15)
Map.addLayerControl()
Map.addLayer(s2_clouds_im.clip(l1), {'bands':['B8','B4','B3'], 'min':0, 'max':3000},'S2')
Map.addLayer(s1_im.clip(l1), {'bands':['VV'], 'min':-15, 'max':15}, 'S1')
Map.addLayer(s2_sample_rects_all.draw(color = 'black'), {}, 'location box')


KeyboardInterrupt



In [ ]:
# # Map the sample locations
# sampmap = geemap.Map()
# # buffer around pts by 64 px

# sampmap.addLayer(hyd_watershed,{},'watershed')
# sampmap.centerObject(hyd_watershed,7)
# sampmap.addLayerControl()
# sampmap.addLayer(s2_clouds_im.clip(s2_sample_rects_map),{'bands':['B8','B4','B3'], 'min':0, 'max':3000}, 's2 fcc')
# sampmap.addLayer(s1_im.clip(s2_sample_rects_map),{'bands':['VV'], 'min':-15, 'max':15}, 's1')
# sampmap.addLayer(ee.Image().paint(s2_sample_rects_map, 1,2),{}, 'sample locations')

## 3. Generate grid, extract data and export

Now loop through each location and:

1. Convert each point location to a rectangular grid of points

  * This is done using the helper functions `rs.get_grid_pts_func`. 

2. Extract timeseries for the entire grid of points from the image collection

  * Each point contains `loc_id`, which is the id of the original point, and `pt_id`, which is the id of each individual gridded point for that location.

3. Export the grid of points and the timeseries of band values

  * These will have filename, e.g., s2_loc_0_pts and s2_loc_0_ts for the points and timeseries files, respectively.


  <!-- * The size of `s2_sample_rect_pts` should be $\textit{square_px}^2\times \textit{numLocations}$. In other words, each sampled location will be associated with a grid of points that is $\textit{square_px}\times \textit{square_px}$. -->



In [71]:
# path for output
output_drive_path = "/content/drive/MyDrive/_Research projects/asiaAg/pyscripts/asiaAg_GEE"

# this returns a function which take a feature as input
get_loc_grid_pts = rs.get_grid_pts_func(s2_clouds_im, square_px, nominal_scale)

# # The following line of code gets the grid of points for all locations at once. But better to do it in the for loop individually for each location.
# s2_sample_rect_pts = s2_sample_locs.map(get_loc_grid_pts).flatten()

# loop through each location and export timeseries
for i in range(s2_sample_locs.size().getInfo()):
  s2_sample_loc = ee.Feature(s2_sample_locs.toList(1,i).get(0))
  loc_id = str(s2_sample_loc.get('loc_id').getInfo())

  pts_filename_prefix = 's2_loc_' + loc_id + '_pts'
  ts_s1_filename_prefix  = 's1_loc_' + loc_id + '_ts'
  ts_s2_filename_prefix  = 's2_loc_' + loc_id + '_ts'
  loc_tif_filename_prefix = 'sentinel_loc_' + loc_id

  # print(type(loc_id))
  # print('loc_id: ' + loc_id + '...\n')

  pts_filepath = os.path.join(output_drive_path, pts_filename_prefix + '.csv')
  ts_s1_filepath  = os.path.join(output_drive_path, ts_s1_filename_prefix + '.csv')
  ts_s2_filepath  = os.path.join(output_drive_path, ts_s2_filename_prefix + '.csv')
  loc_tif_filepath = os.path.join(output_drive_path, loc_tif_filename_prefix + '.tif')

  if not os.path.exists(loc_tif_filepath):
    print('generating tif image ' + loc_tif_filename_prefix + '.tif')
    task_im = ee.batch.Export.image.toDrive(
        image = sentinel_im.select(['B8','B4','B3','B2','clouds','shadows','cloudmask','VV','VH','VV_ep','VH_ep']),
        description = loc_tif_filename_prefix,
        fileNamePrefix = loc_tif_filename_prefix,
        folder = 'asiaAg_GEE',
        region = buffer_rect(s2_sample_loc).geometry(),
        scale = 10,
        maxPixels = 1e7
    )
    task_im.start()

  if os.path.exists(pts_filepath) & os.path.exists(ts_s1_filepath) & os.path.exists(ts_s2_filepath):
    print(pts_filename_prefix + '.csv, ' + 
          ts_s1_filename_prefix + '.csv, and ' + 
          ts_s2_filename_prefix + '.csv exist. Skipping.\n')

  else:

    # get grid points
    loc_grid_pts = get_loc_grid_pts(s2_sample_loc)

    # export grid points
    if not os.path.exists(pts_filepath):
      print('starting task for ' + pts_filename_prefix + '.csv')
      task_pts = ee.batch.Export.table.toDrive(
          collection = loc_grid_pts.select(['loc_id', 'pt_id']),
          description = 'Export pts, loc_id: ' + loc_id,
          folder = 'asiaAg_GEE',
          fileNamePrefix = pts_filename_prefix,
          fileFormat = "CSV")
      task_pts.start()    
      
    # Sentinel 1 timeseries
    if not os.path.exists(ts_s1_filepath):
      print('starting task for ' + ts_s1_filename_prefix + '.csv')

      # some images have HH & HV, others are VV and VH -- therefore extract all bands from all images
      loc_s1_ts = rs.get_pixel_ts_allbands(
          pts_fc = loc_grid_pts, 
          image_collection = s1_ic, 
          ic_property_id = 'system:index', 
          scale = 10)
      task_s1_ts = ee.batch.Export.table.toDrive(
          collection = loc_s1_ts,
          description = 'Export S1 ts, loc_id: ' + loc_id,
          folder = 'asiaAg_GEE',
          selectors = s1_bands + ['loc_id', 'pt_id', 'image_id'],
          fileNamePrefix = ts_s1_filename_prefix,
          fileFormat = "CSV")
      task_s1_ts.start()

    # Sentinel 2 timeseries
    if not os.path.exists(ts_s2_filepath):
      print('starting task for ' + ts_s2_filename_prefix + '.csv')

      loc_s2_ts = rs.get_pixel_timeseries(
          pts_fc = loc_grid_pts, 
          image_collection = s2_clouds_ic, 
          bands = output_bands, 
          ic_property_id = 'system:index', 
          scale = 10)
      
      task_s2_ts = ee.batch.Export.table.toDrive(
          collection = loc_s2_ts,
          description = 'Export S2 ts, loc_id: ' + loc_id,
          folder = 'asiaAg_GEE',
          fileNamePrefix = ts_s2_filename_prefix,
          selectors = output_bands + ['loc_id', 'pt_id', 'image_id'],
          fileFormat = "CSV")
      
      task_s2_ts.start()

s2_loc_0_pts.csv, s1_loc_0_ts.csv, and s2_loc_0_ts.csv exist. Skipping.

generating tif image sentinel_loc_1.tif
s2_loc_1_pts.csv, s1_loc_1_ts.csv, and s2_loc_1_ts.csv exist. Skipping.

generating tif image sentinel_loc_2.tif
s2_loc_2_pts.csv, s1_loc_2_ts.csv, and s2_loc_2_ts.csv exist. Skipping.

generating tif image sentinel_loc_3.tif
s2_loc_3_pts.csv, s1_loc_3_ts.csv, and s2_loc_3_ts.csv exist. Skipping.

generating tif image sentinel_loc_4.tif
s2_loc_4_pts.csv, s1_loc_4_ts.csv, and s2_loc_4_ts.csv exist. Skipping.

generating tif image sentinel_loc_5.tif
s2_loc_5_pts.csv, s1_loc_5_ts.csv, and s2_loc_5_ts.csv exist. Skipping.

generating tif image sentinel_loc_6.tif
s2_loc_6_pts.csv, s1_loc_6_ts.csv, and s2_loc_6_ts.csv exist. Skipping.

generating tif image sentinel_loc_7.tif
s2_loc_7_pts.csv, s1_loc_7_ts.csv, and s2_loc_7_ts.csv exist. Skipping.

generating tif image sentinel_loc_8.tif
s2_loc_8_pts.csv, s1_loc_8_ts.csv, and s2_loc_8_ts.csv exist. Skipping.

generating tif image se

pixel_ts.first().getInfo()
s2_sample_locs.first().getInfo()

Check status of the export task

In [ ]:
# # https://gis.stackexchange.com/questions/377335/getting-the-status-of-google-earth-engine-task
# task.status()
# ee.data.listOperations()[0]
# # ee.data.getTaskStatus('task name')

s2_sample_locs.toList(100).filter(ee.Filter.rangeContains('id',0,1)).getInfo()

# Sentinel 1 data

# Map the data

Generate the map

In [ ]:
Map = geemap.Map()
Map.addLayerControl()
Map.addLayer(s2_clouds_im.clip(hyd_watershed), {'bands': ['B8','B4','B3'], 'min':0, 'max': 3000},'S2 FCC')
Map.centerObject(hyd_watershed, 8)

### Map the resulting cloud mask

In [ ]:
# Map.addLayer(hyd_watershed,{},"Watershed poly")
# Map.centerObject(hyd_watershed, 4)

In [ ]:
# Map.addLayer(s2_clouds_im.clip(hyd_watershed),{'bands':['probability'],'min': 0, 'max': 100},'probability (cloud)')
# Map.addLayer(s2_clouds_im.clip(hyd_watershed).select('clouds').selfMask(),{'bands':['clouds'],'palette': 'e056fd'},'clouds')
# Map.addLayer(s2_clouds_im.clip(hyd_watershed).select('cloud_transform').selfMask(),{'bands':['cloud_transform'],'min': 0, 'max': 1, 'palette': ['white', 'black']},'cloud_transform')
# Map.addLayer(s2_clouds_im.clip(hyd_watershed).select('dark_pixels').selfMask(),{'bands':['dark_pixels'],'palette': 'orange'},'dark_pixels')
# Map.addLayer(s2_clouds_im.clip(hyd_watershed).select('shadows').selfMask(), {'bands':['shadows'], 'palette': 'yellow'},'shadows')
# Map.addLayer(s2_clouds_im.clip(hyd_watershed).select('cloudmask').selfMask(), {'bands':['cloudmask'], 'palette': 'orange'},'cloudmask')

# Extra code

In [ ]:
s2_pt_sample = s2_clouds_im \
  .select(['B8','B4','B3','clouds']) \
  .sample(region = sample_pt, numPixels = 1, scale = 10)
s2_pt_sample.first().getInfo()

{'geometry': None,
 'id': '0',
 'properties': {'B3': 1278, 'B4': 930, 'B8': 3930, 'clouds': 0},
 'type': 'Feature'}

In [ ]:


# sample_pt = ee.Feature(s2_sample_locs.toList(6).get(5)).geometry()
sample_pt = ee.Geometry.Point(coords = [74.0051, 32.4148])

m = geemap.Map()
m.addLayer(sample_rect,{},'rect')

# Map.addLayer(sample_pt,{},'sample_pt')
# Map.addLayer(sample_rect,{},'sample_rect')

In [ ]:
s2_rect = s2_clouds_im \
  .select(['B8','B4','B3','clouds']) \
  .sampleRectangle(region = sample_rect, 
                   defaultValue = 0, 
                   defaultArrayValue = 0)

# s2_rect.getInfo()

In [ ]:
importlib.reload(rs)
# loc_s1_ts = rs.get_pixel_ts_allbands(
#           pts_fc = s2_sample_locs, 
#           image_collection = s1_ic, 
#           ic_property_id = 'system:index', 
#           scale = 10)

# loc_s1_ts.first().getInfo()